In [51]:
import random
import torch
import random
import torch
import pickle
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle


# !pip install tensorflow

In [102]:
#load data from kaggle

import pandas as pd
import re
from sklearn.model_selection import train_test_split
import os
import kagglehub
import zipfile
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from transformers import get_scheduler

# dataset_dir = "./fake_news_dataset"

# if not os.path.exists(dataset_dir):
#     os.makedirs(dataset_dir)

# dataset_name = "emineyetm/fake-news-detection-datasets"
# print(f"Downloading dataset: {dataset_name}")
# os.system(f"kaggle datasets download -d {dataset_name} -p {dataset_dir}")

# zip_file = None
# for file in os.listdir(dataset_dir):
#     if file.endswith(".zip"):
#         zip_file = os.path.join(dataset_dir, file)
#         break

# with zipfile.ZipFile(zip_file, "r") as zip_ref:
#     zip_ref.extractall(dataset_dir)

# fake_path, true_path = None, None
# for root, dirs, files in os.walk(dataset_dir):
#     for file in files:
#         if "Fake.csv" in file:
#             fake_path = os.path.join(root, file)
#         if "True.csv" in file:
#             true_path = os.path.join(root, file)

# fake_df = pd.read_csv(fake_path)
# true_df = pd.read_csv(true_path)

# fake_df = fake_df[['title', 'text', 'subject', 'date']]
# true_df = true_df[['title', 'text', 'subject', 'date']]

# fake_df['label'] = 0
# true_df['label'] = 1

# df = pd.concat([fake_df, true_df]).reset_index(drop=True)

# Download latest version
# path_Fake_News_Classification = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# Download latest version



#  ----- (Covid19)
path = kagglehub.dataset_download("invalizare/covid-19-fake-news-dataset")
dataset_dir = "/root/.cache/kagglehub/datasets/invalizare/covid-19-fake-news-dataset/"

file_name = None
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if "Val.csv" in file:
            file_name = os.path.join(root, file)
            break
df = pd.read_csv(file_name)
df.rename(columns={'tweet': 'text', 'label': 'label'}, inplace=True)
  # ----- (Covid19)


# path_Twitter = kagglehub.dataset_download("sudishbasnet/truthseekertwitterdataset2023")
# dataset_dir_Twitter = "/root/.cache/kagglehub/datasets/sudishbasnet/truthseekertwitterdataset2023/"


# Load the dataset
# df_Fake_News_Classification = pd.read_csv("/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv")

# Dynamically locate the CSV file

# file_name = None
# for root, dirs, files in os.walk(dataset_dir):
#     for file in files:
#         if "Truth_Seeker_Model_Dataset" in file and file.endswith(".csv"):
#             file_name = os.path.join(root, file)
#             break

# df_Twitter = pd.read_csv(file_name)


# Handle missing values
# missing_values = df.isnull().sum()
# print("\nMissing values per column:").  """ For the Twitter file """
# print(missing_values)
# df.dropna(subset=['tweet', 'BinaryNumTarget'], inplace=True)

# missing_values = df.isnull().sum()
# print("\nMissing values per column:")
# print(missing_values)
# df.dropna(subset=['tweet', 'label'], inplace=True)


# df.rename(columns={'tweet': 'text', 'BinaryNumTarget': 'label'}, inplace=True)
# # Check the first few rows of the dataset

# print(df.head())


In [103]:
df.head()

,id,text,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [104]:
print(df.head())
label_mapping = {'fake': 0, 'real': 1}
df['label'] = df['label'].map({'real': 1, 'fake': 0})
print(df.head())
#check what's missing in the data

# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Check how many rows have missing data
rows_with_missing_values = df[df.isnull().any(axis=1)]

# Print out the missing values summary and rows with missing data
print("Missing values per column:\n", missing_values)
print("\nNumber of rows with missing values:", rows_with_missing_values.shape[0])

# Preview the rows with missing values
print("\nRows with missing values:")
print(rows_with_missing_values.head())

   id                                               text label
0   1  Chinese converting to Islam after realising th...  fake
1   2  11 out of 13 people (from the Diamond Princess...  fake
2   3  COVID-19 Is Caused By A Bacterium, Not Virus A...  fake
3   4  Mike Pence in RNC speech praises Donald Trump’...  fake
4   5  6/10 Sky's @EdConwaySky explains the latest #C...  real
   id                                               text  label
0   1  Chinese converting to Islam after realising th...      0
1   2  11 out of 13 people (from the Diamond Princess...      0
2   3  COVID-19 Is Caused By A Bacterium, Not Virus A...      0
3   4  Mike Pence in RNC speech praises Donald Trump’...      0
4   5  6/10 Sky's @EdConwaySky explains the latest #C...      1
Missing values per column:
 id       0
text     0
label    0
dtype: int64

Number of rows with missing values: 0

Rows with missing values:
Empty DataFrame
Columns: [id, text, label]
Index: []


In [105]:
#clean the data of missing values

# Ensure that all values in 'text' are strings
# df['text'] = df['text'].fillna('')  # Fill NaN values with an empty string
# texts = df['text'].astype(str).tolist()  # Convert to list of strings

# map labels to binary values



# Check the first few entries
print(texts[:1])

print(len(texts[0]))

# Split the dataset
# train_texts, test_texts, train_labels, test_labels = train_test_split(
#     df['text'], df['label'], test_size=0.2, random_state=42
print(df['label'])
dataset = df[['text', 'label']]
print(dataset.head())
dataset = shuffle(dataset, random_state=42).reset_index(drop=True)
texts = dataset['text'].tolist()
labels = dataset['label'].tolist()
print('\n', texts[:2], '\n', labels[:2])

# count, sec, thr, you = 0, 0, 0, 0
# new_dic = []
# for i, k in enumerate(texts):
#   if len(k) < 200:
#     thr += 1
#     # if len(k) > 100:
#     #   print(k, labels[i])
#   if k == ' ':
#     count += 1

#   if k[:4] == 'http':
#     you += 1

#   if k not in new_dic:
#     new_dic.append(k)

filtered_texts = []
filtered_labels = []
for t, l in zip(texts, labels):
  if len(t) > 50 and t not in filtered_texts:
    filtered_texts.append(t)
    filtered_labels.append(l)

print(len(filtered_texts))
print(len(filtered_labels))
# print(count, sec, thr, you, len(texts), len(new_dic))

['_New vaccine can help cure coronavirus in three hours._']
55
0       0
1       0
2       0
3       0
4       1
       ..
2135    0
2136    1
2137    0
2138    0
2139    1
Name: label, Length: 2140, dtype: int64
                                                text  label
0  Chinese converting to Islam after realising th...      0
1  11 out of 13 people (from the Diamond Princess...      0
2  COVID-19 Is Caused By A Bacterium, Not Virus A...      0
3  Mike Pence in RNC speech praises Donald Trump’...      0
4  6/10 Sky's @EdConwaySky explains the latest #C...      1

 ['_New vaccine can help cure coronavirus in three hours._', 'For a breakdown of cases by states in real time please see https://t.co/zQrpNeOfet Currently; Lagos- 25 FCT- 6 Ogun- 2 Ekiti- 1 Oyo- 1 Edo- 1 Total: 36 confirmed cases #COVID19Nigeria'] 
 [0, 1]
2099
2099


In [106]:
#tokenize with bert
#mattias says to put this into batches (XXX - ?)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the text, adjust the max_length parameter based on data analysis (XXX)
def encode_text(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='tf')

# Tokenize the text data
#train_encodings = encode_text(texts[:10])
texts_encodings = tokenizer(list(filtered_texts), padding=True, truncation=True, max_length=512, return_tensors='pt')

# Convert labels to tensors
filtered_labels = torch.tensor(list(filtered_labels))

# Check the tokenized data
print(texts_encodings)
print(len(filtered_texts))

{'input_ids': tensor([[  101,  1035,  2047,  ...,     0,     0,     0],
        [  101,  2005,  1037,  ...,     0,     0,     0],
        [  101,  2429,  2000,  ...,     0,     0,     0],
        ...,
        [  101,  1038, 28074,  ...,     0,     0,     0],
        [  101,  2017,  2323,  ...,     0,     0,     0],
        [  101,  2057,  3764,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
2099


In [ ]:
# with open("temp.pkl", "rb") as fOut:
#   lol = pickle.load(fOut)
# # print(lol['sentences'])

In [ ]:
# print(len(lol['sentences']))
# # print(len(lol['sentences'].index))

72134


Unnamed: 0                                                60264
title         Elon Musk’s Tesla Stock Up $2 Billion Since Jo...
text          Although Tesla CEO Elon Musk shocked Silicon V...
label                                                         0
Name: 60264, dtype: object
tensor(0)


In [107]:
import h5py
import numpy as np

input_ids = texts_encodings['input_ids'].numpy()
attention_mask = texts_encodings['attention_mask'].numpy()
token_type_ids = texts_encodings['token_type_ids'].numpy()
labels = np.array(labels)

with h5py.File('Covid19FakeNews.h5', 'w') as hf:
    hf.create_dataset('input_ids', data=input_ids, compression="gzip")
    hf.create_dataset('attention_mask', data=attention_mask, compression="gzip")
    hf.create_dataset('token_type_ids', data=token_type_ids, compression="gzip")
    hf.create_dataset('labels', data=filtered_labels, compression="gzip")
    dt = h5py.string_dtype(encoding='utf-8')
    hf.create_dataset('texts', data=filtered_texts, dtype=dt)


In [108]:


# Open the HDF5 file in read mode
with h5py.File('FakeNewsDetection.h5', 'r') as hf:
    # Load the datasets
    input_ids = np.array(hf['input_ids'])
    attention_mask = np.array(hf['attention_mask'])
    labels = np.array(hf['labels'])

# Check the loaded data
print("Input IDs shape:", input_ids.shape)
print("Attention Mask shape:", attention_mask.shape)
print("Labels shape:", labels.shape)

# Example: Access a single data point
print("First input_ids:", input_ids[10])
print("First attention_mask:", attention_mask[10])
print("First label:", labels[10])
print(labels)


Input IDs shape: (38502, 512)
Attention Mask shape: (38502, 512)
Labels shape: (38502,)
First input_ids: [  101  2899  1006 26665  1007  1011  2047  3933  3099  3782 13144  1010
  2019  2220 10129  1997  1057  1012  1055  1012  2343  1011 11322  6221
  8398  1010  2003  6195  1037  2448  2000  2599  1996  3951  2120  2837
  1010 14955 18291  2080  2988  2006  9432  1012 26665  2071  2025  3202
 12210  1996  3189  1010  1998  4505  2005 13144  2106  2025  3202  6869
  2000  1037  5227  2005  7615  1012 13144  1010  3005  2744  2004  3099
  4515  1999  2254  2760  1010  2018  2042  2877  8398  1521  1055  6653
  2136  2127  1057  1012  1055  1012  3580  2343  1011 11322  3505  7279
  3401  3728  2165  2058  3041  2023  3204  1012  2783 29300  2278  3472
 27788  3401 26927 15878  2271  2038  2042 10410  2000  3710  2004  8398
  1521  1055  2708  1997  3095  2043  2002  4627  2010  2317  2160  2744
  5553  1012  2322  1010  2418  1012 13144  2018  3390  1037  4883  7226
  4077  8398  1999 

Hash comparison passed. Data is identical.
